<a href="https://colab.research.google.com/github/hejazn86/Stock_price_prediction/blob/main/stock_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extracting Stock data from Yahoo API**

In [3]:
#!pip install yfinance
import yfinance as yf

# Define the stock ticker and date range
ticker = 'AAPL'
start_date = '2014-01-01'
end_date = '2024-06-20'

# Fetch historical stock data of specific company (Apple)
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Save the data
stock_data.to_csv('stock_price.csv')
len(stock_data)

[*********************100%%**********************]  1 of 1 completed


2633

**Extracting Stock data using Alpha vantage API**

In [7]:
# Install and load Alpha Vantage API
!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

# Your API key
api_key = 'WL313PFE45NS5WRT'

# Initialize the TimeSeries class
ts = TimeSeries(key=api_key, output_format='pandas')

# Get historical data for a given ticker
ticker = 'AAPL'
data, meta_data = ts.get_daily(symbol=ticker, outputsize='full')

data.to_csv('stock_vantage.csv')
#print(data.head())
len(data)

  Using cached alpha_vantage-2.3.1-py3-none-any.whl (31 kB)


6201

**Data Processing and Feature Engineering**

In [12]:
# Loading the data
import pandas as pd
import numpy as np

stock_data = pd.read_csv('stock_price.csv')

# check for missing values
stock_data.isnull().sum()

# Feature enginerring : extracting the year and the month
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.set_index('Date', inplace=True)
stock_data['Year'] = stock_data.index.year
stock_data['Month'] = stock_data.index.month

# Calculate the Simple Moving average of the stocks
stock_data['SMA_10'] = stock_data['Close'].rolling(window=10).mean()
stock_data['SMA_100'] = stock_data['Close'].rolling(window=100).mean()

# Calculating The Exponential Moving Average
stock_data['EMA_10'] = stock_data['Close'].ewm(span= 10, adjust = False).mean()
stock_data['EMA_100'] = stock_data['Close'].ewm(span=100, adjust=False).mean()

# Calculate the Volatility (Annual)
#stock_data['Range'] = stock_data['High'] - stock_data['Low']
stock_data['Returns'] = stock_data['Close'].pct_change()
stock_data['Volatility'] = stock_data['Returns'].rolling(window = 10).std()* np.sqrt(252)

# Calculate Momentum Indicators: Relative Strength Index (RSI)
def RelativeStrengthIndex (series, period=14):
    delta = series.diff(1) # get the daily price change
    gain = delta.where(delta > 0, 0)                    # create a series with only the positive price changes
    loss = -delta.where(delta < 0, 0)                   # create a series with only the negative price changes
    avg_gain = gain.rolling(window=period).mean()       #calculate the moving average for the positive changes
    avg_loss = loss.rolling(window=period).mean()       # calculate the moving average for the negative changes
    retrnStr = avg_gain / avg_loss                      # calulate the relative strength
    retrnStridx = 100 - (100 / (1 + retrnStr))          # calculate the index
    return retrnStridx

stock_data['RSI_14'] = RelativeStrengthIndex(stock_data['Close'])


#  Calculate Momentum Indicators: Moving Average Convergence Divergence (MACD)
def MovingAverageConvergDiverg (series, shortPeriod=12, longPeriod=26, signalPeriod=9):
  seriesEMA12 = series.ewm(span=shortPeriod, adjust=False).mean()
  seriesEMA26 = series.ewm(span=longPeriod, adjust=False).mean()
  MACD = seriesEMA12 - seriesEMA26
  signalLine = MACD.ewm(span=signalPeriod, adjust=False).mean()
  return MACD, signalLine


MACD, SignalLine = MovingAverageConvergDiverg(stock_data['Close'])
stock_data['MACD'] = MACD
stock_data['SignalLine'] = SignalLine

